In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
import random
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [2]:
corpus = pd.read_csv('corpus.csv')
corpus = corpus.drop_duplicates()
corpus.shape

(4911, 2)

In [3]:
def custom_sentence_splitter(row):
    '''
    method to convert rows into sentences, custom cleaning has been performed
    '''
    pattern = r'(?:[A-Za-z]\.)+'
    sentences = []
    sen = ''
    for w in row.split(" "):
        if w.endswith("."):
            if len(re.findall(r'(?:[A-Za-z]\.)', w)) > 1: 
                # ignoring handling acronym
                pass          
            elif len(re.findall(r'(?:[0-9]\.)', w)) == 1:
                # ignoring numbers
                pass
            else:
                sen += " " + w
                sen = sen.strip(".").strip(" ")
                if len(sen):
                    sentences.append(sen)
                sen = w = ""
        elif "." in w:
            pattern = r'(?:([A-Za-z0-9()\.]{1,})\.([A-Za-z]{2,}))'
            if len(re.findall(pattern=pattern, string=w)):
                sp = w.split(".")
                sen += " " + ".".join(sp[:-1])
                sen = sen.strip(".").strip(" ")
                if len(sen): sentences.append(sen)
                sen = ""
                w = sp[-1]
        sen += " " + w
      
    sen = sen.strip(".").strip(" ")
    if len(sen):
        sentences.append(sen.strip(".").strip(" "))
    return sentences


#### Converting rows to sentences and tagging them

In [4]:

pos_tagged_sentences = []
for i, row in tqdm(corpus.iterrows()):
    for i, sen in enumerate(custom_sentence_splitter(row['Section_Text'])):
        tokens = nltk.word_tokenize(sen)
        tagged_sen = nltk.pos_tag(tokens, )
        pos_tagged_sentences.append({
            "Section_ID": row["Section_ID"],
            "Sentence_ID": "{}_{}".format(row["Section_ID"], i),
            "Sentence": sen,
            "Tagged Sentence": tagged_sen,    
        })

print(len(pos_tagged_sentences))

4911it [00:55, 88.77it/s] 

19254


#### Sample Results

In [5]:
for i in range(10):
    tagged_sentence = random.choice(pos_tagged_sentences)
    print("Example: {}".format(i))
    print("Raw: {}".format(tagged_sentence["Sentence"]))
    print("Tagged: {}".format(tagged_sentence["Tagged Sentence"]))
    print("\n")

Example: 0
Raw: all general allowances of the merged or acquired company must be excluded from the national bank's or federal savings association's eligible credit reserves
Tagged: [('all', 'DT'), ('general', 'JJ'), ('allowances', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('merged', 'VBN'), ('or', 'CC'), ('acquired', 'VBN'), ('company', 'NN'), ('must', 'MD'), ('be', 'VB'), ('excluded', 'VBN'), ('from', 'IN'), ('the', 'DT'), ('national', 'JJ'), ('bank', 'NN'), ("'s", 'POS'), ('or', 'CC'), ('federal', 'JJ'), ('savings', 'NNS'), ('association', 'NN'), ("'s", 'POS'), ('eligible', 'JJ'), ('credit', 'NN'), ('reserves', 'NNS')]


Example: 1
Raw: sponsor means, with respect to a covered fund to serve as a general partner, managing member, or trustee of a covered fund, or to serve as a commodity pool operator with respect to a covered fund as defined in (b)(1)(ii) of this section; in any manner to select or to control (or to have employees, officers, or directors, or agents who constitute) a majorit